In [ ]:
!pip install scikit-plot

In [ ]:
!wget http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip

--2022-01-15 08:14:40--  http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip
Resolving infolab.tamu.edu (infolab.tamu.edu)... 45.55.217.29
Connecting to infolab.tamu.edu (infolab.tamu.edu)|45.55.217.29|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip [following]
--2022-01-15 08:14:40--  https://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip
Connecting to infolab.tamu.edu (infolab.tamu.edu)|45.55.217.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 263659672 (251M) [application/zip]
Saving to: ‘social_honeypot_icwsm_2011.zip’

social_honeypot_icw 100%[===================>] 251.45M   112MB/s    in 2.2s    

2022-01-15 08:14:42 (112 MB/s) - ‘social_honeypot_icwsm_2011.zip’ saved [263659672/263659672]



In [ ]:
!unzip social_honeypot_icwsm_2011.zip

Archive:  social_honeypot_icwsm_2011.zip
   creating: social_honeypot_icwsm_2011/
  inflating: social_honeypot_icwsm_2011/content_polluters.txt  
   creating: __MACOSX/
   creating: __MACOSX/social_honeypot_icwsm_2011/
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters.txt  
  inflating: social_honeypot_icwsm_2011/content_polluters_followings.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters_followings.txt  
  inflating: social_honeypot_icwsm_2011/content_polluters_tweets.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters_tweets.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._legitimate_users.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users_followings.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._legitimate_users_followings.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users_tweets.txt  
  inflating: __MACOSX/soci

In [ ]:
import pandas as pd
import numpy as np
import re, string

from scipy.stats import zscore
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import scikitplot as skplt
import matplotlib.pyplot as plt

In [ ]:
def func(string):
   return pd.Series(string.split(',')).astype(int)
   
dfContent_polluters = pd.read_csv('social_honeypot_icwsm_2011/content_polluters.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"])

dfContent_polluters_followings = pd.read_csv('social_honeypot_icwsm_2011/content_polluters_followings.txt',sep='\t', names=
                 ["UserID","SeriesOfNumberOfFollowings"],converters={"SeriesOfNumberOfFollowings":func},index_col=0)

dfLegitimate_users = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"])

dfLegitimate_users_followings = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users_followings.txt',sep='\t', names=
                 ["UserID","SeriesOfNumberOfFollowings"],converters={"SeriesOfNumberOfFollowings":func},index_col=0)

In [ ]:
def get_user_id(data_frame) :
  df = pd.DataFrame({"UserID":data_frame["UserID"]})
  return df

In [ ]:
data_set_polluters = get_user_id(dfContent_polluters)

data_set_legitimate = get_user_id(dfLegitimate_users)

In [ ]:
data_set_polluters.insert(0,"Class",1)
data_set_legitimate.insert(0,"Class",0)

In [ ]:
data_set_users=pd.concat([data_set_polluters,data_set_legitimate])

In [ ]:
def attributesOfUniqueValues(data_set_users) :
  list=[]
  for col in data_set_users.columns:
    if (data_set_users[col]==data_set_users[col].iloc[0]).all == True:
      list.append(col)
  return data_set_users.drop(list)

In [ ]:
def duplicateRows(data_set_users):
  return data_set_users.drop_duplicates()

In [ ]:
def missingValues(data_set_users):
  valuesReplaced = data_set_users.replace([np.inf,-np.inf],np.nan)
  valuesReplaced.fillna(0,inplace=True)
  return valuesReplaced

In [ ]:
def shiftCol(data_set_users):
  df=data_set_users.drop(["Class"],axis=1)
  # df=df.apply(zscore)
  df["Class"]=data_set_users["Class"]
  return df

In [ ]:
data_set_polluters=attributesOfUniqueValues(data_set_polluters)
data_set_polluters=duplicateRows(data_set_polluters)
data_set_polluters=missingValues(data_set_polluters)
data_set_polluters=shiftCol(data_set_polluters)

In [ ]:
data_set_legitimate=attributesOfUniqueValues(data_set_legitimate)
data_set_legitimate=duplicateRows(data_set_legitimate)
data_set_legitimate=missingValues(data_set_legitimate)
data_set_legitimate=shiftCol(data_set_legitimate)

In [ ]:
export_csv = data_set_users.to_csv (r'data_set_users.csv', index = None, header=True)

In [ ]:
export_csv = data_set_polluters.to_csv (r'data_set_polluters.csv', index = None, header=True)

In [ ]:
export_csv = data_set_legitimate.to_csv (r'data_set_legitimate.csv', index = None, header=True)

In [ ]:
dfContent_polluters_followings

,SeriesOfNumberOfFollowings
UserID,
6301,0 3269 1 3310 2 3339 3 338...
10836,0 1949 1 1963 2 1963 3 196...
10997,0 1119 1 1119 2 999 3 99...
633293,0 2174 1 2651 2 2676 3 267...
717883,0 7731 1 7737 2 7737 3 774...
...,...
173715049,0 63 1 118 2 178 3 208 dtype: int64
173718053,0 83 1 122 2 157 3 194 dtype: int64
173721018,0 67 1 106 2 145 3 193 dtype: int64


In [ ]:
# X=data_set_users.drop(['Class'],axis=1)
# X=X.values
# Y=data_set_users["Class"].values
# train, test, target, trueResult = train_test_split(X, Y, test_size=0.3, random_state=1)

In [ ]:
!pip install networkx

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
import tweepy as tweepy


In [ ]:
# authenticaing twitter api user
consumer_key  = 'd9Ksoz6Wb1jD0mqbW8rjaSNb7'
consumer_secret = 'pHXnVSJeLbOxaYlbOR7BWFdDNhZSF6IzegZV87qUSUqy6Qe8qG'
access_token = '3648603434-dGRu1nHet22tdoYeqaAGoN8MyZrNw9oXZQvGZUD'
access_token_secret = 'PZ8pcQBCb5zVPLRQNVQZc3Yzi0rz1wPef6O7RO7gzcvOf'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# ID = 11142774119
# retweets_list = api.retweets(ID)
  
# # printing the screen names of the retweeters
# for retweet in retweets_list:
#     print(retweet.user.screen_name)

In [ ]:
# data_set_users

In [ ]:
# userListpolluters = []
# for index , row in data_set_polluters.iterrows():
#     try:
#         print(row['UserID'])
#         u=api.get_user(row['UserID'])
#         userListpolluters.append(u.id_str)
#     except Exception:
#           data_set_polluters.drop(index)
#           pass

In [ ]:
# export_csv = data_set_polluters.to_csv (r'dfUserspolluters.csv', index = None, header=True)

In [ ]:
# dfUserspolluters = pd.DataFrame(userListpolluters)

In [ ]:
# dfUserspolluters
# export_csv = dfUserspolluters.to_csv (r'dfUserspolluters2.csv', index = None, header=True)

In [ ]:
# userListpolluters = []
# for x in data_set_polluters['UserID']:
#     try:
#         u=api.get_user(x)
#         userListpolluters.append(u.id_str)
#     except Exception:
#           pass

In [ ]:
# userListlegitimate = []
# for x in data_set_legitimate['UserID']:
#     try:
#         u=api.get_user(x)
#         userListlegitimate.append(u.id_str)
#     except Exception:
#           pass

In [ ]:
# dfUserspolluters = pd.DataFrame(userListpolluters, columns = ['UserID'])
# dfUserspolluters.insert(0,"Class",1)

# dfUserslegitimate = pd.DataFrame(userListlegitimate, columns = ['UserID'])
# dfUserslegitimate.insert(0,"Class",0)

In [ ]:
# export_csv = dfUserspolluters.to_csv (r'dfUserspolluters.csv', index = None, header=True)
# export_csv = dfUserslegitimate.to_csv (r'dfUserslegitimate.csv', index = None, header=True)

In [ ]:
# dfUserspolluters=dfUserspolluters.drop(["Class"],axis=1)
# dfUserspolluters.shape

In [ ]:
# dfUserslegitimate=dfUserslegitimate.drop(["Class"],axis=1)
# dfUserslegitimate.shape

In [ ]:
# Create a mostly empty data frame,
# and write it to a CSV file.
# df = pd.DataFrame(columns=['ID','followers'])
# df.to_csv('followers.csv', index=False,header=False)

# # Our function
# def save_followers(fid, followers):
#     df = pd.DataFrame([[fid, followers]])
#     with open('followers.csv', 'a') as f:
#         df.to_csv(f,header=False, index=False)

In [ ]:

#sorting the top friends
# dfExample = dfUserslegitimate[:5]
# dfExample


In [ ]:
# for seed_id in dfUserslegitimate['UserID']:
# print("60742090   follwers ")
# friendids = api.friends_ids(id=60742090)
# print("60742090   finish ")
# for second_id in dfUserslegitimate['UserID']:
#     if (second_id in friendids) :
#         print(" 60742090   ---- ", second_id)
# for third_id in dfUserspolluters['UserID']:
#     if (third_id in friendids) :
#         print(" 60742090   ---- ", second_id)
# #     top5_friends = api.friends(id=seed_id)[:5]
# #     for friend in top5_friends:
# #         friendsList.append(friend.screen_name)
# #         print(friend.screen_name)
# save_followers(user_id, friend_id)

In [ ]:
# friends = api.friends_ids(id="20293152")

In [ ]:
# for friend in api.friends(id="20293152")[:5]:
#    friends.append(friend.screen_name)
#    print(friend.screen_name)

In [ ]:
# friendsList = []
# for seed_id in dfExample['UserID']:
#     top5_friends = api.friends(id=seed_id)[:5]
#     for friend in top5_friends:
#         friendsList.append(friend.screen_name)
#         print(friend.screen_name)
    # for user_id, friend_id in [(user_id, friend_id) for user_id in [int(seed_id)] 
    #     for friend_id in top5_friends]:
    #     save_followers(user_id, friend_id)

In [ ]:
# for user_id, friend_id in [(user_id, friend_id) for user_id in [int(seed_id)] 
#     for friend_id in top5_friends]:
#     save_followers(user_id, friend_id)

In [ ]:
# userList = []
# for x in data_set_users['UserID']:
#     try:
#         u=api.get_user(x)
#         userList.append(u.id_str)
#     except Exception:
#           pass

In [ ]:
# dfUsers = pd.DataFrame(userList, columns = ['UserID'])

In [ ]:
# dfUsers

In [ ]:
# export_csv = dfUsers.to_csv (r'dfUsers.csv', index = None, header=True)